# Предсказание эмоции в реальном времени

In [1]:
%cd
try:
    import cv2
except ModuleNotFoundError:
    ! pip install opencv-python
    import cv2
    
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    ! pip install matplotlib
    import matplotlib.pyplot as plt
    
import numpy as np
from copy import deepcopy
import urllib.request
import tensorflow as tf
import os

C:\Users\user


## Импорт модели:

In [2]:
# Устанавливаем модуль import_ipynb и импортируем его.
# Он необходим для импорта файлов .ipynb.
try:
    import import_ipynb
except:
    ! pip install import-ipynb
    import import_ipynb

# Если проект отсутствует, клонируем его с Github. Далее из проекта нам понадобиться файл 
# импорта модели - import_model.ipynb, расположенный в корне проекта.
# Импортируем из него класс Model
try:
    from skillbox_thesis_project.import_model import Model
except ModuleNotFoundError:
    ! git clone https://github.com/GrinkoL/skillbox_thesis_project.git
    from skillbox_thesis_project.import_model import Model

importing Jupyter notebook from C:\Users\user\skillbox_thesis_project\import_model.ipynb


In [3]:
# Инициализируем модель предсказания эмоции.
model = Model()

Downloading...
From: https://drive.google.com/uc?id=1-HDQxtpZKGuKWQhsiDANyc41SL_CwHdU
To: C:\Users\user\model.zip
1.34GB [12:06, 1.84MB/s]


## Подключение к камере:

In [4]:
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1080) # Устанавливаем ширину кадра (по умолчанию - 640 пикселей)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) # Устанавливаем высоту кадра (по умолчанию - 480 пикселей)

if not cam.isOpened():
    print("Не удалось открыть камеру")
else:
    print("Камера запущена")

Камера запущена


## Инициализаци детектора лиц:

In [5]:
# Загружаем файл классического детектора лиц основанный на методе Виолы-Джонсона и признаках Хаара.
if os.name == 'nt': # если Windows
    url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml'
    filename = 'haarcascade_frontalface_default.xml'
    urllib.request.urlretrieve(url, filename)
elif os.name == 'posix': # если Linux
    !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
else:
    raise Exception('Этот ноутбук должен быть запущен либо на Linux либо на Windows (7 и старше) операционной системе')

# Иницилизируем детектор:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Запуск видео и отображение имоции в реальном времени

In [6]:
# В бесконечном цикле считываем кадр с камеры и выводим его, используя cv2.imshow()
# корректная остановка окна с камерой произойдет, когда мы нажмем q на клавиатуре 

while(True):
    # получение кадра с камеры
    ret, frame = cam.read()
    # получение координат лиц на кадре
    grayscale_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(grayscale_image, 1.3, 5)
    # Запускаем обработку кадров моделью только в случае детекции лиц.
    # Если лица обнаружены, то faces указывает на данные типа np.ndarray, в противном случае -
    # на пустой tuple.
    if isinstance(faces, np.ndarray) : 
        one_face = faces[0] # будем учитывать только первое детектируемое лицо
        x, y, w, h = one_face
        #  Подготавливаем изображение для модели: выризаем лицо, меняем очерёдность цветовых слоём.
        face_boundingbox_bgr = frame[y:y + h, x:x + w]
        face_boundingbox_rgb = cv2.cvtColor(face_boundingbox_bgr, cv2.COLOR_BGR2RGB)
        # Запуск предсказания - получение эмоции на текущем кадре.
        emotion = model.predict(face_boundingbox_rgb.astype('float32'))
        # Рисуем на кадре bounding-box лица.
        frame = cv2.rectangle(frame, (x, y), (x + h, y + w), (0,255,0), 3)
        # Добавляем текст - распознанную эмоцию
        frame = cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    # Отображаем кадр
    cv2.imshow("facial emotion recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break